## Intro to workflows for data analysis, using snakemake
### Authors: Meghan Correa and Ming Tang
### Bioinformatics Coffee Hour - May 26, 2020

Modified from https://github.com/ctb/2019-snakemake-ucdavis

No license; the below content is under CC0. (Do with it what you will, and I hope it's useful!)

### Introduction and thoughts

Hi! I'm Meghan Correa, I a member of the Software Operations team at FAS informatics. I support data analysis for the Bauer core among other things.

This 25min teaser meant to show you how to get started using snakemake.

Note, the only way you'll really learn to do all of this is by applying it to your own research and spending time on it :).

### Why use a workflow management tool?

* Dependency management
* Reentrancy - start back up where you left off
* Reusable
* Documented
* Portable

There are many to choose from:
https://github.com/pditommaso/awesome-pipeline

FAS Informatics choose to use snakemake for the post sequencing pipeline and other workflows mainly because it is written in python and is popular in the bioinformatics community.

### Tasks we are going to do

Inside the `data` folder, there are 4 fastq.gz files. We will use `Snakemake` do `fastqc` on each fastq files and compile multiple html files into a single one using `multiQC`.

## Software we're going to use

We're going to be using [conda](https://conda.io/en/latest/) and [snakemake](https://snakemake.readthedocs.io/en/stable/), a well as packages from [bioconda](https://bioconda.github.io). If you wanted to run all of this on your own computer, you'll need to follow the bioconda install instructions.

We'll be implementing a short read quality check pipeline, using [fastqc](https://www.bioinformatics.babraham.ac.uk/projects/fastqc/) and [multiqc](https://multiqc.info/). No worries if you don't know what any of this means, it's not super critical to the snakemake side of things :)

You can see the full set of installed software requirements for python in `environment.yml` file and installs from apt repository in `apt.txt` in the binder folder of the repository.

**Note we have installed all the required tools in the binder container**

## Download some data

Execute the below cell to fetch some data.

In [ ]:
curl -L https://github.com/ctb/2019-snakemake-ucdavis/raw/9db09bc0b6a3469f8a0d4996d4b2995bf36e5d27/data/0Hour_001_1.fq.gz > data/0Hour_001_1.fq.gz
curl -L https://github.com/ctb/2019-snakemake-ucdavis/raw/9db09bc0b6a3469f8a0d4996d4b2995bf36e5d27/data/6Hour_001_1.fq.gz > data/6Hour_001_1.fq.gz
curl -L https://github.com/ctb/2019-snakemake-ucdavis/raw/9db09bc0b6a3469f8a0d4996d4b2995bf36e5d27/data/0Hour_001_2.fq.gz > data/0Hour_001_2.fq.gz
curl -L https://github.com/ctb/2019-snakemake-ucdavis/raw/9db09bc0b6a3469f8a0d4996d4b2995bf36e5d27/data/6Hour_001_2.fq.gz > data/6Hour_001_2.fq.gz

## Running snakemake!

### Getting started - your first Snakefile

Open the `Snakemake` file with a text editor and paste the below into it:

In [ ]:
rule fastqc_a_file:
  shell:
    "fastqc data/0Hour_001_1.fq.gz"

(I suggest copy/pasting this into the Snakefile.)

remember to click `File` --> `Save File`.

In [ ]:
snakemake

and you should see:
```
Building DAG of jobs...
Using shell: /bin/bash
Provided cores: 1
Rules claiming more threads will be scaled down.
...
Approx 95% complete for 0Hour_001_1.fq.gz
Analysis complete for 0Hour_001_1.fq.gz
[Wed Feb 27 13:09:51 2019]
Finished job 0.
1 of 1 steps (100%) done
Complete log: /home/jovyan/.snakemake/log/2019-02-27T130941.260352.snakemake.log
```

and there will be two new files,

Points to make:
* the snakemake configuration file is by default called `Snakefile`

### Updating the Snakefile to track inputs and outputs

At the moment this is basically just a shell script with extra syntax... what's the point?

Well, shell scripts will rerun the command every time you run the file, even if there's no reason to do so because the file hasn't changed but with snakemake, you can annotate the rule with input and output files!

**Digression:** This is particularly important for large or long workflows, where you're dealing with 10s to 100s of files that may take hours to days to process! It can be hard to figure out which files to rerun, but (spoiler alert) snakemake can really help you do this!

Change your snakefile to look like this:


In [ ]:
rule fastqc_a_file:
  input:
    "data/0Hour_001_1.fq.gz"
  output:
    "data/0Hour_001_1_fastqc.html",
    "data/0Hour_001_1_fastqc.zip"
  shell:
    "fastqc data/0Hour_001_1.fq.gz"

here, we've annotated the rule with the required
**input** file, as well as the expected **output** files.

Question: how do we know what the output files are?

Now run:

In [ ]:
snakemake

and you should see:
```
Building DAG of jobs...
Nothing to be done.
Complete log: /home/jovyan/.snakemake/log/2019-02-27T132031.813143.snakemake.log
```

What happened??

snakemake looked at the file, saw that the output files existed, and figured out that it didn't need to do anything!

### Forcibly re-running things

You can tell snakemake to run all the rules no matter what with `-F`:

In [ ]:
snakemake -F

You can also remove an output file and it will automatically re-run:

In [ ]:
rm data/*.html
snakemake

note that you don't need to remove *all* the output files to rerun a command - just remove *one* of them.

You can *also* update the timestamp on an *input* file, and snakemake will figure out that the output file is older than the input file, and rerun things.

In [ ]:
touch data/*.fq.gz
snakemake

Another usefule snakemake option is the --dryrun option, this will show you what snakemake would do but won't actually run any of the rules.  I highly recommend running this to check that was will be run is what you expect.

In [ ]:
snakemake --dryrun

### Multiple rules

Let's add a rule to run fastqc on a second file:

In [ ]:
rule fastqc_a_file:
  input:
    "data/0Hour_001_1.fq.gz"
  output:
    "data/0Hour_001_1_fastqc.html",
    "data/0Hour_001_1_fastqc.zip"
  shell:
    "fastqc data/0Hour_001_1.fq.gz"

rule fastqc_a_file2:
  input:
    "data/6Hour_001_1.fq.gz"
  output:
    "data/6Hour_001_1_fastqc.html",
    "data/6Hour_001_1_fastqc.zip"
  shell:
    "fastqc data/6Hour_001_1.fq.gz"

In [ ]:
snakemake

Now, if you run this, the Right Thing won't happen: snakemake will do nothing.  Why?

Well, snakemake only runs the *first* rule in a Snakefile, by default. You can give a rule name on the command line, if you like, **or** you can tell snakemake what output file(s) you want. 

But the conventional way to do this in snakemake is to add an "all" rule as the first rule, let's try it. 


### A first refactoring: adding a better default rule

Let's start refactoring (cleaning up) this Snakefile.

First, let's add a rule at the top:

In [ ]:
rule all:
  input:
    "data/0Hour_001_1_fastqc.html",
    "data/6Hour_001_1_fastqc.html"

rule fastqc_a_file:
  input:
    "data/0Hour_001_1.fq.gz"
  output:
    "data/0Hour_001_1_fastqc.html",
    "data/0Hour_001_1_fastqc.zip"
  shell:
    "fastqc data/0Hour_001_1.fq.gz"

rule fastqc_a_file2:
  input:
    "data/6Hour_001_1.fq.gz"
  output:
    "data/6Hour_001_1_fastqc.html",
    "data/6Hour_001_1_fastqc.zip"
  shell:
    "fastqc data/6Hour_001_1.fq.gz"

In [ ]:
snakemake

This is a blank rule that gathers together all of the various files you want produced, and says "hey, snakemake, I depend on all of these files for my input - make them for me!"

Let's run snakemake again and now you should see the second fastqc command run, with the appropriate output files!

Note that snakemake only runs the second rule, because it looks at the output files and sees that the first file you wanted, `0Hour_001_1_fastqc.html` already exists!

This is a blank rule that gathers together all of the various files you want produced, and says "hey, snakemake, I depend on all of these files for my input - make them for me!"

### A second refactoring: doing a bit of templating

There's a lot of repetition in each of these rules. Let's collapse it down a little bit by replacing the filename in the fastqc command with a magic variable, `{input}`.

In [ ]:
rule all:
  input:
    "data/0Hour_001_1_fastqc.html",
    "data/6Hour_001_1_fastqc.html"

rule fastqc_a_file:
  input:
    "data/0Hour_001_1.fq.gz"
  output:
    "data/0Hour_001_1_fastqc.html",
    "data/0Hour_001_1_fastqc.zip"
  shell:
    "fastqc {input}"

rule fastqc_a_file2:
  input:
    "data/6Hour_001_1.fq.gz"
  output:
    "data/6Hour_001_1_fastqc.html",
    "data/6Hour_001_1_fastqc.zip"
  shell:
    "fastqc {input}"

This all works as before, but now the rule is a bit more generic and will work with any input file. Sort of.

### Refactoring 3: templating output files, too

The output filenames ALSO depend on the input file names in some way - specifically, fastqc replace part of the filename with `_fastqc.html` and `_fastqc.zip` to make its two output files.

Let's rewrite the rule using some snakemake pattern matching:

In [ ]:
rule all:
  input:
    "data/0Hour_001_1_fastqc.html",
    "data/6Hour_001_1_fastqc.html"

rule fastqc_a_file:
  input:
    "{filename}.fq.gz"
  output:
    "{filename}_fastqc.html",
    "{filename}_fastqc.zip"
  shell:
    "fastqc {input}"

rule fastqc_a_file2:
  input:
    "{filename}.fq.gz"
  output:
    "{filename}_fastqc.html",
    "{filename}_fastqc.zip"
  shell:
    "fastqc {input}"

What we've done here is tell snakemake that anytime we say we *want* a file that ends with `_fastqc.html`, it should look for a file that ends in `.fq.gz` and then run `fastqc` on it.

If you look at the rule above, they are now identical rules! 

Let's remove one, to get a trimmer, leaner, and above all *functional* snakefile:

In [ ]:
rule all:
  input:
    "data/0Hour_001_1_fastqc.html",
    "data/6Hour_001_1_fastqc.html"

rule fastqc_a_file:
  input:
    "{filename}.fq.gz"
  output:
    "{filename}_fastqc.html",
    "{filename}_fastqc.zip"
  shell:
    "fastqc {input}"

Note that the variable name in input and output does not have to be "filename", it can be anything as long as you're consistant.  So let's run snakemake again.

In [ ]:
rm data/*.html
snakemake

### Building out the workflow

So, we've gotten fastqc sorted out. What's next?

Let's add in a new rule - multiqc, to summarize our fastqc results.

multiqc takes a directory name under which there are one or more fastqc reports, and then summarizes them.

Running it on the command line,

In [ ]:
multiqc data

you can see that it creates two outputs, `multiqc_report.html` and the directory `multiqc_data/` which contains a bunch of files. Let's create a snakemake rule for this; add:

In [ ]:
rule all:
  input:
    "data/0Hour_001_1_fastqc.html",
    "data/6Hour_001_1_fastqc.html"

rule fastqc_a_file:
  input:
    "{filename}.fq.gz"
  output:
    "{filename}_fastqc.html",
    "{filename}_fastqc.zip"
  shell:
    "fastqc {input}"

rule run_multiqc:
  input:
    "data/0Hour_001_1_fastqc.html",
    "data/6Hour_001_1_fastqc.html",
  output:
    "multiqc_report.html",
    directory("multiqc_data")
  shell:
    "multiqc data/"

In [ ]:
snakemake

This ...doesn't really do what we want, for a few reasons.

First of all, the output of run_multiqc is not specified in the all rule so snakemake doesn't look for a rule to create this. 

Second of all, `multiqc_report.html` already exists, so snakemake won't run the rule. 

Let's fix the first two things first:

* add `multiqc_report.html` to the inputs for the first all.
* then remove `multiqc_report.html` and re-run snakemake.

Your snakefile should look like:

In [ ]:
rule all:
  input:
    "data/0Hour_001_1_fastqc.html",
    "data/6Hour_001_1_fastqc.html",
    "multiqc_report.html"

rule fastqc_a_file:
  input:
    "{filename}.fq.gz"
  output:
    "{filename}_fastqc.html",
    "{filename}_fastqc.zip"
  shell:
    "fastqc {input}"

rule run_multiqc:
  input:
    "data/0Hour_001_1_fastqc.html",
    "data/6Hour_001_1_fastqc.html",
  output:
    "multiqc_report.html",
    directory("multiqc_data")
  shell:
    "multiqc data/"

In [ ]:
rm multiqc_report.html
snakemake

Yay, that seems to work!

Points to make:

* other than the first rule, rules can be in any order
* the rule name doesn't really matter, it's mostly for debugging. It just needs to be "boring" (text, underscores, etc. only)


But providing input files explicitly to the multiqc rule is not great because those are the same files we have in the all rule and to add new files we'd now have to add them to two places. 

We can fix the first issue by using **variables**. 

To use variables, let's make a Python list at the very top, containing all of our expected output files from fastqc:

In [ ]:
fastqc_output = ["data/0Hour_001_1_fastqc.html", "data/6Hour_001_1_fastqc.html",
  "data/0Hour_001_2_fastqc.html", "data/6Hour_001_2_fastqc.html"]

and modify the `all` and `multiqc` rules to contain this list. The final snakefile looks like this:

In [ ]:
fastqc_output = ["data/0Hour_001_1_fastqc.html", "data/6Hour_001_1_fastqc.html",
  "data/0Hour_001_2_fastqc.html", "data/6Hour_001_2_fastqc.html"]

rule all:
  input:
    fastqc_output,
    "multiqc_report.html"

rule fastqc_a_file:
  input:
    "{filename}.fq.gz"
  output:
    "{filename}_fastqc.html",
    "{filename}_fastqc.zip"
  shell:
    "fastqc {input}"

rule run_multiqc:
  input:
    fastqc_output
  output:
    "multiqc_report.html",
    directory("multiqc_data")
  shell:
    "multiqc data/"

In [ ]:
snakemake

### Refactoring this to make it slightly more concise --

We've got one more redundancy in this file - the `fastqc_output` is listed in the `all` rule, but you don't need it there! Why?

Well, `multiqc_report.html` is already in the all rule, and the multiqc rule depends on `fastqc_output`, so `fastqc_output` already needs to be created to satisfy the all rule, so... specifying it in the all rule is redundant! And you can remove it!

(It's not a big deal and I usually leave it in. But I wanted to talk about dependencies!)

The Snakefile now looks like this:

In [ ]:
fastqc_output = ["data/0Hour_001_1_fastqc.html", "data/6Hour_001_1_fastqc.html",
  "data/0Hour_001_2_fastqc.html", "data/6Hour_001_2_fastqc.html"]

rule all:
  input:
    "multiqc_report.html"

rule fastqc_a_file:
  input:
    "{filename}.fq.gz"
  output:
    "{filename}_fastqc.html",
    "{filename}_fastqc.zip"
  shell:
    "fastqc {input}"

rule run_multiqc:
  input:
    fastqc_output
  output:
    "multiqc_report.html",
    directory("multiqc_data")
  shell:
    "multiqc data/"

and we can rerun it from scratch by doing:

In [ ]:
rm data/*.html multiqc_report.html
snakemake

### Recap

So, we've put all this work into making this snakefile with its input rules and its output rules... and there are a lot of advantages to our current approach already! Let's list a few of them --

* we've completely automated our analysis!
* we can easily add new data files into fastqc and multiqc!
* we can rerun things easily, and (even better) by default only things that *need* to be run will be run.
* the snakefile is actually pretty reusable - we could drop this into a new project, and, with little effort, run all of these things on new data!

## More advanced snakemake

### Running things in parallel

You can use `snakemake --cores 4` to run four jobs in parallel. If you are running on a cluster then --cores becomes the number of jobs that can be submitted at a time.  

### Specifying software required for a rule

You can specify software on a per-rule basis! This is really helpful when you have incompatible software requirements for different rules, or want to run on a cluster, or just want to pin your snakemake workflow to a specific version.

For example, if you create a file `env_fastqc.yml` with the following content,

In [ ]:
channels:
  - bioconda
  - defaults
  - conda-forge
dependencies:
  - fastqc==0.11.8

and then change the fastqc rule to look like this:

you can now run snakemake like so,

and for that rule, snakemake will install just that software, with the specified version.

This aids in reproducibility, in addition to the practical advantages of isolating software installs from each other.

(You can also do this with docker and singularity containers, too!)

### Running on a cluster

You can specify a cluster submit command:

snakemake --cluster sbatch -j 32

You can also specify per rule cluster config in a file (example below):

snakemake --cluster sbatch --cluster-config cluster-config.json -j 32 


In [ ]:
{
   "default":{
      "time":"12:00:00",
      "nodes":1,
      "cores":24,
      "mem":100,
      "partition":"shared",
      "job-name":"{rule}",
      "output":"log/{rule}-%j.out",
      "error":"log/{rule}-%j.err"
   },
   "fastqc":{
      "time":"4-12:00:00",
      "mem":32000
   },
   "multiqc":{
      "mem":64000
   }
}

Or you can use snakemake profiles and bundle cluster config files in a directory:

snakemake --profile slurm -j 32

https://snakemake.readthedocs.io/en/v5.18.1/executing/cli.html#profiles

### Adding in some Python...

You can add in some Python to load in the input files, like so:

### Go forth and workflow!